# Basic Big Query Operations - Loading Regression Data 

<img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">

... some semi advanced Code to manage Big Query Tables including partitions. Though these sample files are so samll partitions are not really needed this might com in handy at another time.



### GCP Locations
https://cloud.google.com/about/locations

https://cloud.google.com/bigquery/docs/locations#supported_locations


---

by Markus Lauber (https://medium.com/@mlxl)

https://yam-united.telekom.com/profile/markus-lauber/

In [ ]:
from google.colab import auth
auth.authenticate_user()
project_id = 'de123456-user-prd-1'
dataset_id = 'xgb_regression_project'
bucket_id = 'gs://mybucket/'

from google.cloud import bigquery
import pandas as pd
from pandas_gbq import to_gbq

# Initialize the BigQuery client
client = bigquery.Client(project=project_id)

### Rename Columns that have - (hyphen) in their name to _ (underscore)

Standard big query tables do not seem to like special characters in column names. So best to remove hyphens (in this case)

In [ ]:
def rename_columns_underscore_and_number(project_id, dataset_id, table_re_name):
    """
    Args:
      project_id:
      dataset_id:
      table_re_name:
    """
    # Initialize a client
    client = bigquery.Client(project=project_id)

    # Get the table schema
    table_re_ref = client.dataset(dataset_id).table(table_re_name)
    table_re = client.get_table(table_re_ref)

    # Prepare SQL to rename columns with hyphens or starting with a number
    queries = []
    for field in table_re.schema:
        new_name = field.name

        # Replace hyphens with underscores
        if '-' in field.name:
            new_name = new_name.replace('-', '_')

        # Check if the column name starts with a digit
        if field.name[0].isdigit():
            new_name = f"v_{new_name}"

        # If a change is required, prepare the SQL statement
        if new_name != field.name:
            print("Old: ", field.name, " | New: ", new_name)
            sql = f"ALTER TABLE `{project_id}.{dataset_id}.{table_re_name}` RENAME COLUMN `{field.name}` TO `{new_name}`;"
            queries.append(sql)

    # Execute the queries
    for query in queries:
        client.query(query)

    print("Column renaming completed.")


The Bucket where the Parquet files are being stored. You can obviously use other datasets and methods to load them

gs://mybucket/dataset_regression_80.parquet

In [ ]:
# Define your table ID (which includes dataset)
table_name = 'regression_train'
table_id = f"{project_id}.{dataset_id}.{table_name}"

# Set up the configuration for the load job
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.PARQUET,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # This line replaces the data instead of appending
)

# URI of the Parquet file in Google Cloud Storage
# https://github.com/ml-score/knime_meets_python/tree/main/machine_learning/regression
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview
uri = 'gs://mybucket/dataset_regression_80.parquet'

# API request - starts the load job
load_job = client.load_table_from_uri(
    uri,
    table_id,
    job_config=job_config
)

# Wait for the job to complete
load_job.result()

print("Job finished.")


Job finished.


In [ ]:
# Example usage to remove unwanted characters in column names:
rename_columns_underscore_and_number(project_id, dataset_id, table_name)

Old:  1stFlrSF  | New:  v_1stFlrSF
Old:  2ndFlrSF  | New:  v_2ndFlrSF
Old:  3SsnPorch  | New:  v_3SsnPorch
Column renaming completed.


In [ ]:
# sometimes it seems on go is not enough to capture all of the instances. So just run this a second or third time
rename_columns_underscore_and_number(project_id, dataset_id, table_name)

Column renaming completed.


In [ ]:
# Fetch the table
table = client.get_table(table_id)  # Make sure to provide the full table ID here

```Python
@bpd.remote_function(bpd.DataFrame, str, str)
def add_constant_column(df, column_name, constant_value):
    df[column_name] = constant_value
    return df
```


In [ ]:
type(table)

google.cloud.bigquery.table.Table

In [ ]:
# Extract the schema and format it into the desired Python code structure
schema_code_snippets = []
for field in table.schema:
    description = field.description if field.description else ''
    schema_line = f'bigquery.SchemaField("{field.name}", "{field.field_type}", description="{description}")'
    schema_code_snippets.append(schema_line)

In [ ]:
# Extract the field names and join them with commas
field_names = ", ".join(field.name for field in table.schema)

# Print the comma-separated field names
print("Fields: ", field_names)

Fields:  MSSubClass, MSZoning, LotFrontage, LotArea, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, YearBuilt, YearRemodAdd, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, Heating, HeatingQC, CentralAir, Electrical, v_1stFlrSF, v_2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, KitchenQual, TotRmsAbvGrd, Functional, Fireplaces, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PavedDrive, WoodDeckSF, OpenPorchSF, EnclosedPorch, v_3SsnPorch, ScreenPorch, PoolArea, PoolQC, Fence, MiscFeature, MiscVal, MoSold, YrSold, SaleType, SaleCondition, Target, row_id


In [ ]:
# Combine the snippets into the full schema definition
schema_code = "[\n    " + ",\n    ".join(schema_code_snippets) + "\n]"

# Print the generated code
print("# Define the table schema with various data types")
print("schema = ", schema_code)

# Define the table schema with various data types
schema =  [
    bigquery.SchemaField("MSSubClass", "INTEGER", description=""),
    bigquery.SchemaField("MSZoning", "STRING", description=""),
    bigquery.SchemaField("LotFrontage", "INTEGER", description=""),
    bigquery.SchemaField("LotArea", "INTEGER", description=""),
    bigquery.SchemaField("Street", "STRING", description=""),
    bigquery.SchemaField("Alley", "STRING", description=""),
    bigquery.SchemaField("LotShape", "STRING", description=""),
    bigquery.SchemaField("LandContour", "STRING", description=""),
    bigquery.SchemaField("Utilities", "STRING", description=""),
    bigquery.SchemaField("LotConfig", "STRING", description=""),
    bigquery.SchemaField("LandSlope", "STRING", description=""),
    bigquery.SchemaField("Neighborhood", "STRING", description=""),
    bigquery.SchemaField("Condition1", "STRING", description=""),
    bigquery.SchemaField("Condition2", "STRING", description=""),
    bigquery.SchemaField("

### Use the schema you just extracted

You can now use the schema to create a (new) partitioned Big Query table (just to demonstrate how this is done). You also might add descriptions. Currently you can only have one partitioning column and they only can be long/integers or date fields.

Note: I added the category field and use the "MSSubClass" as a partition where I would accept integers from 1 to 500.
"Neighborhood" is used for clustring which means the table wil be sorted to make access easier

In [ ]:
# Define your dataset and table ID
table_id = 'regression_data'

# TIMESTAMP,

# bigquery.SchemaField("category", "STRING", description="")

# Define the table schema with various data types
schema =  [
    bigquery.SchemaField("MSSubClass", "INTEGER", description=""),
    bigquery.SchemaField("MSZoning", "STRING", description=""),
    bigquery.SchemaField("LotFrontage", "INTEGER", description=""),
    bigquery.SchemaField("LotArea", "INTEGER", description=""),
    bigquery.SchemaField("Street", "STRING", description=""),
    bigquery.SchemaField("Alley", "STRING", description=""),
    bigquery.SchemaField("LotShape", "STRING", description=""),
    bigquery.SchemaField("LandContour", "STRING", description=""),
    bigquery.SchemaField("Utilities", "STRING", description=""),
    bigquery.SchemaField("LotConfig", "STRING", description=""),
    bigquery.SchemaField("LandSlope", "STRING", description=""),
    bigquery.SchemaField("Neighborhood", "STRING", description=""),
    bigquery.SchemaField("Condition1", "STRING", description=""),
    bigquery.SchemaField("Condition2", "STRING", description=""),
    bigquery.SchemaField("BldgType", "STRING", description=""),
    bigquery.SchemaField("HouseStyle", "STRING", description=""),
    bigquery.SchemaField("OverallQual", "INTEGER", description=""),
    bigquery.SchemaField("OverallCond", "INTEGER", description=""),
    bigquery.SchemaField("YearBuilt", "INTEGER", description=""),
    bigquery.SchemaField("YearRemodAdd", "INTEGER", description=""),
    bigquery.SchemaField("RoofStyle", "STRING", description=""),
    bigquery.SchemaField("RoofMatl", "STRING", description=""),
    bigquery.SchemaField("Exterior1st", "STRING", description=""),
    bigquery.SchemaField("Exterior2nd", "STRING", description=""),
    bigquery.SchemaField("MasVnrType", "STRING", description=""),
    bigquery.SchemaField("MasVnrArea", "INTEGER", description=""),
    bigquery.SchemaField("ExterQual", "STRING", description=""),
    bigquery.SchemaField("ExterCond", "STRING", description=""),
    bigquery.SchemaField("Foundation", "STRING", description=""),
    bigquery.SchemaField("BsmtQual", "STRING", description=""),
    bigquery.SchemaField("BsmtCond", "STRING", description=""),
    bigquery.SchemaField("BsmtExposure", "STRING", description=""),
    bigquery.SchemaField("BsmtFinType1", "STRING", description=""),
    bigquery.SchemaField("BsmtFinSF1", "INTEGER", description=""),
    bigquery.SchemaField("BsmtFinType2", "STRING", description=""),
    bigquery.SchemaField("BsmtFinSF2", "INTEGER", description=""),
    bigquery.SchemaField("BsmtUnfSF", "INTEGER", description=""),
    bigquery.SchemaField("TotalBsmtSF", "INTEGER", description=""),
    bigquery.SchemaField("Heating", "STRING", description=""),
    bigquery.SchemaField("HeatingQC", "STRING", description=""),
    bigquery.SchemaField("CentralAir", "STRING", description=""),
    bigquery.SchemaField("Electrical", "STRING", description=""),
    bigquery.SchemaField("v_1stFlrSF", "INTEGER", description=""),
    bigquery.SchemaField("v_2ndFlrSF", "INTEGER", description=""),
    bigquery.SchemaField("LowQualFinSF", "INTEGER", description=""),
    bigquery.SchemaField("GrLivArea", "INTEGER", description=""),
    bigquery.SchemaField("BsmtFullBath", "INTEGER", description=""),
    bigquery.SchemaField("BsmtHalfBath", "INTEGER", description=""),
    bigquery.SchemaField("FullBath", "INTEGER", description=""),
    bigquery.SchemaField("HalfBath", "INTEGER", description=""),
    bigquery.SchemaField("BedroomAbvGr", "INTEGER", description=""),
    bigquery.SchemaField("KitchenAbvGr", "INTEGER", description=""),
    bigquery.SchemaField("KitchenQual", "STRING", description=""),
    bigquery.SchemaField("TotRmsAbvGrd", "INTEGER", description=""),
    bigquery.SchemaField("Functional", "STRING", description=""),
    bigquery.SchemaField("Fireplaces", "INTEGER", description=""),
    bigquery.SchemaField("FireplaceQu", "STRING", description=""),
    bigquery.SchemaField("GarageType", "STRING", description=""),
    bigquery.SchemaField("GarageYrBlt", "INTEGER", description=""),
    bigquery.SchemaField("GarageFinish", "STRING", description=""),
    bigquery.SchemaField("GarageCars", "INTEGER", description=""),
    bigquery.SchemaField("GarageArea", "INTEGER", description=""),
    bigquery.SchemaField("GarageQual", "STRING", description=""),
    bigquery.SchemaField("GarageCond", "STRING", description=""),
    bigquery.SchemaField("PavedDrive", "STRING", description=""),
    bigquery.SchemaField("WoodDeckSF", "INTEGER", description=""),
    bigquery.SchemaField("OpenPorchSF", "INTEGER", description=""),
    bigquery.SchemaField("EnclosedPorch", "INTEGER", description=""),
    bigquery.SchemaField("v_3SsnPorch", "INTEGER", description=""),
    bigquery.SchemaField("ScreenPorch", "INTEGER", description=""),
    bigquery.SchemaField("PoolArea", "INTEGER", description=""),
    bigquery.SchemaField("PoolQC", "STRING", description=""),
    bigquery.SchemaField("Fence", "STRING", description=""),
    bigquery.SchemaField("MiscFeature", "STRING", description=""),
    bigquery.SchemaField("MiscVal", "INTEGER", description=""),
    bigquery.SchemaField("MoSold", "INTEGER", description=""),
    bigquery.SchemaField("YrSold", "INTEGER", description=""),
    bigquery.SchemaField("SaleType", "STRING", description=""),
    bigquery.SchemaField("SaleCondition", "STRING", description=""),
    bigquery.SchemaField("Target", "INTEGER", description=""),
    bigquery.SchemaField("row_id", "STRING", description=""),
    bigquery.SchemaField("category", "STRING", description="Indicating if Test or Training")
]
# Define integer range partitioning settings for the "education_num" column, could also be year or any other
range_partitioning = bigquery.RangePartitioning(
    field="MSSubClass",
    range_=bigquery.PartitionRange(
        start=1,
        end=500,
        interval=1
    )
)

# Define table reference
table_ref = client.dataset(dataset_id).table(table_id)

# Define table with schema and partitioning settings
table = bigquery.Table(table_ref, schema=schema)
table.range_partitioning = range_partitioning

# Specify clustering fields directly in the table object
table.clustering_fields = ["Neighborhood"]

# Create the table
client.create_table(table)


Table(TableReference(DatasetReference('de123456-user-prd-1', 'xgb_regression_project'), 'regression_data'))

### Import the TEST data also

In [ ]:
# Define your table ID (which includes dataset)
table_name = 'regression_test'
table_id = f"{project_id}.{dataset_id}.{table_name}"
# table_id = '{}.{}.census_train'.format(project_id, dataset_name)

# Set up the configuration for the load job
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.PARQUET,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # This line replaces the data instead of appending
)

# URI of the Parquet file in Google Cloud Storage
# https://github.com/ml-score/knime_meets_python/tree/main/machine_learning/regression
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview
uri = 'gs://mybucket/dataset_regression_20.parquet'

# API request - starts the load job
load_job = client.load_table_from_uri(
    uri,
    table_id,
    job_config=job_config
)

# Wait for the job to complete
load_job.result()

print("Job finished.")


Job finished.


In [ ]:
rename_columns_underscore_and_number(project_id, dataset_id, table_name)

Old:  1stFlrSF  | New:  v_1stFlrSF
Old:  2ndFlrSF  | New:  v_2ndFlrSF
Old:  3SsnPorch  | New:  v_3SsnPorch
Column renaming completed.


In [ ]:
rename_columns_underscore_and_number(project_id, dataset_id, table_name)

Column renaming completed.


## Insert TRAIN and TEST into the (partitioned) target table

In [ ]:
# Define your source and destination tables
source_table = 'regression_test'
source_table_id = f"{project_id}.{dataset_id}.{source_table}"

destination_table = 'regression_data'
destination_table_id = f"{project_id}.{dataset_id}.{destination_table}"

print("Source: ", source_table_id, " | Target table: ", destination_table_id)

Source:  de123456-user-prd-1.xgb_regression_project.regression_test  | Target table:  de123456-user-prd-1.xgb_regression_project.regression_data


## Make sure the table is empty (also handy if you have to set up the table again)

In [ ]:
# Create a query to truncate the table
query = f"TRUNCATE TABLE `{destination_table_id}`"

# Execute the query
query_job = client.query(query)
query_job.result()  # Wait for the job to complete

print(f"Table {destination_table_id} has been truncated.")

Table de123456-user-prd-1.xgb_regression_project.regression_data has been truncated.


## INSERT TEST data first

In [ ]:
# SQL query to append data from source to destination
query = f"""
INSERT INTO `{destination_table_id}`
SELECT *
, '{source_table}' AS category
FROM `{source_table_id}`
WHERE row_id  NOT IN (SELECT DISTINCT row_id FROM `{destination_table_id}` WHERE category = '{source_table}')
"""

# Execute the query
query_job = client.query(query)  # Make an API request.

# Wait for the job to complete
query_job.result()

print("Data has been appended successfully.")


Data has been appended successfully.


## now insert TRAIN data

In [ ]:
# Define your source and destination tables
source_table = 'regression_train'
source_table_id = f"{project_id}.{dataset_id}.{source_table}"

In [ ]:
# SQL query to append data from source to destination
query = f"""
INSERT INTO `{destination_table_id}`
SELECT *
, '{source_table}' AS category
FROM `{source_table_id}`
WHERE row_id  NOT IN (SELECT DISTINCT row_id FROM `{destination_table_id}` WHERE category = '{source_table}')
"""
# Execute the query
query_job = client.query(query)  # Make an API request.

# Wait for the job to complete
query_job.result()

print("Data has been appended successfully.")


Data has been appended successfully.


In [ ]:
# SQL Query - modify the query to match your table and desired aggregation
query = f"""
SELECT
  category,
  AVG(Target) as avg_target,
  COUNT(*) AS Anzahl
FROM
  `{destination_table_id}` AS t1
  GROUP BY category
  ORDER BY category
"""

# Execute the query and load results into a DataFrame
query_job = client.query(query)  # Run the query
df = query_job.to_dataframe()  # Convert the results into a pandas DataFrame

# Display the DataFrame
print(df)

           category     avg_target  Anzahl
0   regression_test  177287.848375     277
1  regression_train  181771.945900    1183
